In [11]:
#Importing the modules
import numpy as np
import dpdata
import os
import matplotlib.pyplot as plt

In [13]:
# Function definitions:

def get_atoms_info(line):
    natoms = int(line.strip().split()[4])
    nspecies = int(line.strip().split()[1])
    return natoms, nspecies

def get_energy(line):
    return float(line.strip().split()[2])

def get_lattice_vectors(lines, i):
    x = list(map(float, lines[i].strip().replace('[', '').replace(']', '').split()))
    y = list(map(float, lines[i+1].strip().replace('[', '').replace(']', '').split()))
    z = list(map(float, lines[i+2].strip().replace('[', '').replace(']', '').split()))

    box = np.zeros((3,3))
    box[0, :] = np.array(x)
    box[1, :] = np.array(y)
    box[2, :] = np.array(z)
    box_corr = box.T
    x_corr = box_corr[0, :].tolist()
    y_corr = box_corr[1, :].tolist()
    z_corr = box_corr[2, :].tolist()

    return x_corr + y_corr + z_corr

def get_volume(line):
    return float(line.strip().split()[4])

def get_stress_tensor(lines, i, volume):
    x = list(map(float, lines[i].strip().replace('[', '').replace(']', '').split()))
    y = list(map(float, lines[i+1].strip().replace('[', '').replace(']', '').split()))
    z = list(map(float, lines[i+2].strip().replace('[', '').replace(']', '').split()))
    xx = [i*volume for i in x]
    yy = [i*volume for i in y]
    zz = [i*volume for i in z]
    return xx + yy + zz

def get_coordinates(lines, natoms, i):
    clines = lines[i:i+natoms]
    coord = np.array([list(map(float, line.strip().split()[2:5])) for line in clines]).reshape(-1)
    allatoms = [line.strip().split()[1] for line in clines]
    return coord, allatoms

def get_forces(lines, natoms, i):
    flines = lines[i:i+natoms]
    force = np.array([list(map(float, line.strip().split()[2:5])) for line in flines]).reshape(-1)
    return force

def automapping(allatoms):
    """
    Use the atom_types in the input and create type.raw
    and type_map.raw files
    """
    # Identifying unique atom types in the simulation:
    #atom_types = list(set(allatoms)); 

    # Assigning integer numbers to unique elements
    mapping = {atom: i for i, atom in enumerate(atom_types)}; 

    # Replacing elements with their mapping
    atm_mapped = [mapping[atom] for atom in allatoms]

    # Writing type_map.raw:
    with open("type.raw", 'w') as f:
        for value in atm_mapped:
            f.write(f"{value}\n")

    # Writing type.raw:
    with open("type_map.raw", 'w') as f:
        for i in atom_types:
            f.write(f"{i}\n")
    return atm_mapped

def all_convert(filename, frame_sep):
    """
    Collects the Force, enegy, box, virial datas from JDFTXoutput file
    """
    with open(filename) as file:
        lines = file.readlines()

        Etot, Force, Stress, Lattice, Coord = [], [], [], [], []

        for i, line in enumerate(lines):
            if "Initialized" in line:
                natoms, nspecies = get_atoms_info(line)
            if "F =    " in line:
                Etot.append(get_energy(line))
            if "# Lattice vectors:" in line:
                Lattice.append(get_lattice_vectors(lines, i+2))
            if "unit cell volume =" in line:
                volume = get_volume(line)
#             if "# Stress tensor in Cartesian coordinates" in line:
#                 Stress.append(get_stress_tensor(lines, i+1, volume))
            if "Ionic positions in cartesian coordinates" in line:
                coord, allatoms = get_coordinates(lines, natoms, i+1)
                Coord.append(coord)
            if "Forces in Cartesian coordinates" in line:
                Force.append(get_forces(lines, natoms, i+1))
    
    # Converting the list to array
    Etot = np.array(Etot)
    Lattice = np.array(Lattice)
    Coord = np.array(Coord)
    Force = np.array(Force)
    
    # Sampling the AIMD data:
    AIMD_frame_no = len(Etot) // frame_sep
    Frame_index = np.arange(AIMD_frame_no)*frame_sep

    Etot = Etot[Frame_index]
    Lattice = Lattice[Frame_index]
    Coord = Coord[Frame_index]
    Force = Force[Frame_index]

    # Shuffling the data:
    print(">Force, Energy, Coordinate, Lattice converted, shuffled, and saved")
    permuted_indices = np.random.permutation(len(Etot))
    Etot = np.array(Etot)[permuted_indices]*ene_conv
    Lattice = np.array(Lattice)[permuted_indices]*len_conv
    Coord = np.array(Coord)[permuted_indices]*len_conv
    Force = np.array(Force)[permuted_indices]*force_conv
#     Stress = np.array(Stress)[permuted_indices]*ene_conv
    
    # Energy correction:
    Etot = Etot - Energy_atom(allatoms)

    # Saving the data:
    np.savetxt("energy.raw", Etot)
    np.savetxt("box.raw", Lattice)
    np.savetxt("coord.raw", Coord)
    np.savetxt("force.raw", Force)
#     np.savetxt("virial.raw", Stress)
    
    return Etot, Force, Coord, Lattice, allatoms, natoms, nspecies

def Energy_atom(allatoms):
    """ Calculate the energy of atoms to be subtracted from bulk """
    # Assigning integer numbers to unique elements
    mapping = {atom: i for i, atom in enumerate(atom_types)}; 
    # Replacing elements with their mapping
    atm_mapped = [mapping[atom] for atom in allatoms]
    nCo = atm_mapped.count(0)
    nSi = atm_mapped.count(1)
    nO = atm_mapped.count(2)
    print('Co:', nCo, 'Si:', nSi,'O:', nO)
    E_atom = nCo*E_Co + nSi*E_Si + nO*E_O
    return E_atom

In [14]:
atom_types = ['Co', 'Si', 'O']  # CHANGE HERE, Add the types of elements that you use 
                                # in your project even if certain datas has only few elements

# Conversion from jdftx to deepmd or lammps
ene_conv = 27.2114              # hartree to eV conversion
len_conv = 0.529177             # bohr to Angstrom conversion
force_conv = ene_conv/len_conv  # E = fd, so f = E/d 

# Single atom energies:
E_Co = -148.5065685 * ene_conv # hartree to eV
E_Si = -3.80475254 * ene_conv # hartree to eV
E_O = -15.8884064 * ene_conv # hartree to eV

In [30]:
file_AIMD = "CoSi_NPT_bulk_300k.jdftxout"

with open(file_AIMD) as file:
    lines = file.readlines()
    total_lines = len(lines); print("Total lines:",  total_lines)
    dump_index = []                    # Extracting the total energy
    
    # Getting the dump indexing:
    for i, line in enumerate(lines):
        if "Dumping 'md.ionpos' ... done" in line:
            dump_index.append(i)
    
    print(len(dump_index))
    start_ind = dump_index[0:-1]
    end_ind = dump_index[1:]
    # print(len(start_ind), len(end_ind))
    total_frame = len(start_ind)
    cframe = 0

    Etot, Force, Stress, Lattice, Coord, dump_index = [], [], [], [], [], []

    for i in range(total_frame):                          # Looping over all frames
    #     print(i)
        frame = lines[start_ind[i]:end_ind[i]]            # Get the frame
        for linee in frame:
            if "Converged" in linee:
                cframe = cframe + 1
                good_frame = frame

                for j, nline in enumerate(good_frame):
                    
                    if "Initialized" in nline:
                        natoms = int(nline.strip().split()[4])
                        nspecies = int(nline.strip().split()[1])
                        
                    if "F =    " in nline:
                        Etot.append(float(nline.strip().split()[2]))
                    
                    if "# Lattice vectors:" in nline:
                            Lattice.append(get_lattice_vectors(good_frame, j+2))
                    
                    if "unit cell volume =" in nline:
                            volume = get_volume(nline)
                    
                    if "Ionic positions in cartesian coordinates" in nline:
                            coord, allatoms = get_coordinates(good_frame, natoms, j+1)
                            Coord.append(coord)
                    if "Forces in Cartesian coordinates" in nline:
                            Force.append(get_forces(good_frame, natoms, j+1))
                            
    print("Converged frames:", cframe, "Unconverged frames:", total_frame-cframe)


Total lines: 618511
2580
Converged frames: 2577 Unconverged frames: 2


In [31]:
print(len(Etot), len(Lattice), len(Force))


2577 2577 2577


In [33]:
start_ind[0:3]

[713, 955, 1197]

In [34]:
end_ind[0:3]

[955, 1197, 1445]

In [37]:
lines[713:955+1]

["Dumping 'md.ionpos' ... done\n",
 "Dumping 'md.lattice' ... done\n",
 '\n',
 '#--- Lowdin population analysis ---\n',
 '# oxidation-state Co -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858 -0.858\n',
 '# oxidation-state Si +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984 +0.984\n',
 '\n',
 '\n',
 'Computing DFT-D3 correction:\n',
 '# coordination-number Co 9.377 9.389 9.388 9.371 9.392 9.377 9.382 9.388 9.382 9.387 9.377 9.384 9.387 9.379 9.395 9.378 9.384 9.380 9.376 9.375 9.388 9.379 9.397 9.398 9.381 9.377 9.387 9.385 9.406 9.368 9.371 9.394\n',
 '# coordination-number Si 10.217 10.235 10.245 10.235 10.242 10.223 10.228 10.221 10.231 10.239 10.231 10